# VGGNet (VGG-16)

## Imports

In [1]:
import tensorflow as tf
from tensorflow import keras 
from keras.models import Sequential, Model 
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

## Model

In [2]:
def add_conv_block(model, filters, num_layers=2):
    for _ in range(num_layers):
        model.add(Conv2D(filters, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, strides=2))
    return model 

def add_dense_block(model, units, activation, dropout_rate=None):
    model.add(Dense(units, activation=activation))
    if dropout_rate:
        model.add(Dropout(dropout_rate))
    return model

def vgg_16_model(input_shape=(200, 200, 3)):
    model = Sequential()
    # Convolutional Layers
    model.add(Conv2D(64, 3, activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, strides=2))
    
    model = add_conv_block(model, 128)
    model = add_conv_block(model, 256)
    model = add_conv_block(model, 512)
    model = add_conv_block(model, 512)

    model.add(Flatten())
    # Fully Connected Layers
    model = add_dense_block(model, 4096, 'relu')
    model = add_dense_block(model, 4096, 'relu')
    model = add_dense_block(model, 1, 'sigmoid')

    return model

model = vgg_16_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 200, 200, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 100, 100, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 50, 50, 128)       0         
 g2D)                                                   

I do not dare run this again.
Accidentally did it, 1 epoch time = 5 hours 31 minutes

## Using Transfer Learning
It is too scary to imagine my laptop trying to train any model with more than 3 layers on a dataset this big let alone a ConvNet this big with or without transfer learning.

In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

for layer in base_model.layers:
    layer.trainable = False

model_2 = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  
])

model_2.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model_2.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 6, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 18432)             0         
                                                                 
 dense_3 (Dense)             (None, 512)               9437696   
                                                                 
 dense_4 (Dense)             (None, 1)                 513       
                                                                 
Total params: 24152897 (92.14 MB)
Trainable params: 9438209 (36.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
